# Estudio de las Ventas de una Ferretería
## NB5: Scraper Tasa de Cambio Dólar

>Autor: Diana Chacón Ocariz

## Contexto:

En los últimos años Venezuela ha sufrido de hiperinflación, lo que dificulta el estudio de datos monetarios en Bs (Bolívares, moneda nacional venezolana). Además, en octubre del 2021 se produjo una reconversión monetaria y se le eliminaron 6 ceros a la moneda.

Por eso se suelen transformar los montos a una moneda más estable, principalmente el dólar. 

En este NB leemos la URL https://monitordolarvenezuela.com/historial#2a para extraer el histórico de las tasas de cambio diarias de Bs a USD utilizando la librería *BeautifulSoup*.

El histórico es almacenado en un archivo CSV que luego es utilizado para transformar los montos monetarios.

In [1]:
import requests
from bs4 import BeautifulSoup
# Para saber el encoding utilizado por el servidor
import chardet

# Leer la página del URL y devolver un objeto BeautifulSoup

def devolver_soup(url):
    # Eliminamos el '/' del final, si existe
    # Da error 403 si se deja
    if url[len(url)-1] == '/':
        url = url[:len(url)-1]
    
    # Obtenemos la página
    try:
        resp = requests.get(url)
    except Exception as e:
        print('Error obteniendo la página. Excepción ',e)
        return url
    
    if resp.status_code != 200:
        print('Error obteniendo la página. Status Code', resp.status_code)
        return url
    
    # Para saber el encoding utilizado por el servidor
    # print(chardet.detect(resp.content))
    
    # Para especificar el tipo de encoding y 
    # obtener correctamente los caracteres acentuados
    resp.encoding = "utf-8"
    contenido = resp.content
    soup = BeautifulSoup(contenido, 'lxml')    

    return soup

In [2]:
import re

# Función general que extrae datos numéricos 
# de una cadena de caracteres. Devuelve un float

# Elimina los "'" y los puntos de mil
# Reemplaza las "," decimales por "."
# Extrae una cifra y la devuelve como float
def extraer_numero(cadena):
    
    result = cadena.replace("'", '').replace('.', '').replace(',', '.')
    
    result = re.findall(r'-?\d+\.?\d*', result)[0]
    
    return float(result)
 

In [3]:
# Extraemos los datos del dólar

URL='https://monitordolarvenezuela.com/historial#2a'

soup = devolver_soup(URL)

# Extraemos la tabla y sus datos
datos = soup.find('table', attrs={'class':'zui-table'}).find_all('tr')

Error obteniendo la página. Excepción  ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


TypeError: find() takes no keyword arguments

In [ ]:
# Extraemos la información de cada línea de la tabla
result = []

for dato in datos:

  dic = {}
  
  tasas = dato.find_all('td')

  if tasas:

    dic['fecha'] = tasas[0].text.strip()
    dic['baja'] = extraer_numero(tasas[1].text.strip())
    dic['alta'] = extraer_numero(tasas[2].text.strip())

    result.append(dic)
    

In [ ]:
# Guardamos el resultado en un archivo CSV
import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd()
FILE= 'datos/in/tasa_dolar.csv'

In [ ]:
df = pd.DataFrame(result)
df.sample(15)


In [ ]:
df.to_csv(f"{BASE_DIR / FILE}", sep=';', index=False)